In [7]:
import tweepy
import pandas as pd

# Claves de la app (las obtienes en developer.x.com)
api_key = "b6kglbl5HV57Pne0FsJF093dR"
api_secret = "E7SLODvuKqZazmBeEc9JlP5Qnc5VgYkcuHwomcGYvQhPuM9hGZ"
access_token = "1988649363288297472-PJcov7FaKIXdGzcHUOWsYvzUmk2Qmi"
access_secret = "brVO46wcVk9EvPvoSBeMZBaWCmh7KzTyNgWxxnLJTrWgZ"
bearer_token = "AAAAAAAAAAAAAAAAAAAAANeq5QEAAAAAgaAEOWsbHdNEk%2F%2BcVIIsQXplK1c%3DPM9QbU4Cc3otvRoLnjHx17QvTc4BukKE6yW2YxhsSv9QdItx7U"

client = tweepy.Client(bearer_token=bearer_token)

query = '("manifestación" OR "protesta" OR "bloqueo" OR "paro") AND ("Uruapan" OR "alcalde Uruapan" OR "presidente municipal Uruapan" OR "Héctor") AND "Michoacán" lang:es -is:retweet place_country:MX'
tweets = client.search_recent_tweets(query=query, max_results=100)

data = []
for tweet in tweets.data:
    data.append(tweet.text)

df = pd.DataFrame(data, columns=["texto"])
df.to_csv("corpus_twitter.csv", index=False)

print("✅ Corpus guardado en corpus_twitter.csv")


TooManyRequests: 429 Too Many Requests
Too Many Requests

In [14]:
# scrape_fb_group_playwright.py
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup
import json
import time

FACEBOOK_EMAIL = "yisuscolombia123@gmail.com"  # usa cuenta secundaria
FACEBOOK_PASSWORD = "yisuscolombiano123"
GROUP_URL = "https://www.facebook.com/groups/2406893716312968"
OUTPUT = "posts_paro_tec.jsonl"

SCROLL_PAUSES = 3   # segundos entre scrolls
MAX_SCROLLS = 100   # cuántos scrolls hacer (ajusta)


def parse_posts_from_html(html):
    soup = BeautifulSoup(html, "lxml")
    posts = []

    # Los selectores cambian; esto es un punto de partida
    for article in soup.find_all("div", {"role": "article"}):
        # intenta extraer texto y timestamp
        text_el = article.find("div", string=True)
        text = text_el.get_text(" ", strip=True) if text_el else ""
        time_el = article.find("abbr")
        ts = time_el.get("data-utime") if time_el and time_el.get("data-utime") else None

        # intenta obtener post url o id
        link = article.find("a", href=True)
        post_url = link["href"] if link else None

        posts.append({
            "text": text,
            "timestamp": ts,
            "post_url": post_url
        })

    return posts


def main():
    seen = set()

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False)  # Puedes usar headless=True, pero a veces Facebook detecta headless
        context = browser.new_context()
        page = context.new_page()

        # LOGIN
        page.goto("https://www.facebook.com/login", timeout=60000)
        page.fill('input[name="email"]', FACEBOOK_EMAIL)
        page.fill('input[name="pass"]', FACEBOOK_PASSWORD)
        page.click('button[name="login"]')
        page.wait_for_load_state("networkidle")
        time.sleep(3)

        # Ir al grupo
        page.goto(GROUP_URL, timeout=60000)
        page.wait_for_load_state("networkidle")
        time.sleep(3)

        # Scroll y colecta HTML en cada paso
        for i in range(MAX_SCROLLS):
            time.sleep(2)  # Espera 2 segundos a que termine de cargar
            try:
                page.wait_for_selector('div[role="article"]', timeout=10000)
            except Exception:
                print(f"[scroll {i+1}] No se encontraron artículos, continuando...")
                continue

            html = page.content()
            posts = parse_posts_from_html(html)

            new = 0
            with open(OUTPUT, "a", encoding="utf-8") as f:
                for post in posts:
                    key = (post.get("post_url") or post.get("text")[:120])
                    if key in seen:
                        continue
                    seen.add(key)
                    f.write(json.dumps(post, ensure_ascii=False) + "\n")
                    new += 1

            print(f"[scroll {i+1}/{MAX_SCROLLS}] nuevos: {new}")

            # Scroll down
            page.evaluate("window.scrollBy(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSES)

        browser.close()

    print("✅ Terminado.")


if __name__ == "__main__":
    main()


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [11]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import json
import time

FACEBOOK_EMAIL = "example"           # usa cuenta secundaria
FACEBOOK_PASSWORD = "example"
GROUP_URL = "https://www.facebook.com/groups/2406893716312968"
OUTPUT = "posts_paro_tec.jsonl"
SCROLL_PAUSES = 3   # segundos entre scrolls
MAX_SCROLLS = 50     # reduce un poco para probar primero

def parse_posts_from_html(html):
    soup = BeautifulSoup(html, "lxml")
    posts = []
    for article in soup.find_all("div", {"role": "article"}):
        text_el = article.find("div", string=True)
        text = text_el.get_text(" ", strip=True) if text_el else ""
        time_el = article.find("abbr")
        ts = time_el.get("data-utime") if time_el and time_el.get("data-utime") else None
        link = article.find("a", href=True)
        post_url = link["href"] if link else None
        posts.append({"text": text, "timestamp": ts, "post_url": post_url})
    return posts

async def main():
    seen = set()
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # LOGIN
        await page.goto("https://www.facebook.com/login", timeout=60000)
        await page.fill('input[name="email"]', FACEBOOK_EMAIL)
        await page.fill('input[name="pass"]', FACEBOOK_PASSWORD)
        await page.click('button[name="login"]')
        await page.wait_for_load_state("networkidle")
        await asyncio.sleep(3)

        # IR AL GRUPO
        await page.goto(GROUP_URL, timeout=60000)
        await page.wait_for_load_state("networkidle")
        await asyncio.sleep(3)

        # SCROLL Y SCRAPE
        for i in range(MAX_SCROLLS):
            await asyncio.sleep(2)
            try:
                await page.wait_for_selector('div[role="article"]', timeout=10000)
            except:
                pass
            html = await page.content()
            posts = parse_posts_from_html(html)
            new = 0
            with open(OUTPUT, "a", encoding="utf-8") as f:
                for post in posts:
                    key = (post.get("post_url") or post.get("text")[:120])
                    if key in seen:
                        continue
                    seen.add(key)
                    f.write(json.dumps(post, ensure_ascii=False) + "\n")
                    new += 1
            print(f"[scroll {i+1}/{MAX_SCROLLS}] nuevos: {new}")
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight);")
            await asyncio.sleep(SCROLL_PAUSES)

        await browser.close()
    print("✅ Terminado")

# En Jupyter usamos await directamente (no asyncio.run)
await main()


NotImplementedError: 

In [ ]:
import feedparser
import schedule
import time
import json
from datetime import datetime, timedelta

RSS_FEEDS = [
    "https://www.mimorelia.com/rss",
    "https://www.quadratin.com.mx/feed/",
    "https://informatey.com/feed/",
    "https://www.itmorelia.edu.mx/feed/"
]

# Palabras clave para filtrar noticias
PALABRAS_CLAVE = [
    "morelia",
    "michoacán",
    "uruapan",
    "manifestación",
    "protesta",
    "violencia",
    "tecnológico de morelia",
    "itm",
    "ingeniería"
]

def fetch_news(days_back=2):
    """Obtiene noticias recientes de varios RSS"""
    noticias = []
    limite = datetime.now() - timedelta(days=days_back)

    for url in RSS_FEEDS:
        print(f"📡 Leyendo RSS: {url}")
        feed = feedparser.parse(url)

        for item in feed.entries:
            try:
                fecha = None
                if hasattr(item, "published_parsed"):
                    fecha = datetime(*item.published_parsed[:6])
                elif hasattr(item, "updated_parsed"):
                    fecha = datetime(*item.updated_parsed[:6])
                else:
                    continue

                if fecha < limite:
                    continue

                noticia = {
                    "titulo": item.title,
                    "link": item.link,
                    "fecha": fecha.strftime("%Y-%m-%d %H:%M:%S"),
                    "descripcion": getattr(item, "summary", "")
                }

                noticias.append(noticia)

            except Exception as e:
                print("❌ Error procesando noticia:", e)

    return noticias


def filtrar(noticias, palabras):
    """Filtra noticias por palabras clave"""
    filtradas = []
    for n in noticias:
        texto = (n["titulo"] + " " + n["descripcion"]).lower()
        if any(p.lower() in texto for p in palabras):
            filtradas.append(n)
    return filtradas


def guardar_json(data):
    """Guarda resultados en un archivo JSON"""
    with open("noticias.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print(f"💾 Guardado {len(data)} noticias en noticias.json")


def tarea_automatica():
    print("\n==============================")
    print("⏳ Ejecutando scraping automático:", datetime.now())
    print("==============================")

    noticias = fetch_news(days_back=3)
    print(f"📥 Total noticias encontradas: {len(noticias)}")

    filtradas = filtrar(noticias, PALABRAS_CLAVE)
    print(f"🔎 Noticias filtradas: {len(filtradas)}")

    guardar_json(filtradas)

    print("✅ Completado.\n")


# Programa de ejecución automática cada X tiempo
schedule.every(30).minutes.do(tarea_automatica)   # <-- cambia aquí el intervalo

print("🤖 Scraper automático iniciado...")
print("Se ejecutará cada 30 minutos.\n")

tarea_automatica()  # Primera ejecución inmediata

while True:
    schedule.run_pending()
    time.sleep(1)


🤖 Scraper automático iniciado...
Se ejecutará cada 30 minutos.


⏳ Ejecutando scraping automático: 2025-11-18 10:09:06.316134
📡 Leyendo RSS: https://www.mimorelia.com/rss
📡 Leyendo RSS: https://www.quadratin.com.mx/feed/
📡 Leyendo RSS: https://informatey.com/feed/
📡 Leyendo RSS: https://www.itmorelia.edu.mx/feed/
📥 Total noticias encontradas: 10
🔎 Noticias filtradas: 10
💾 Guardado 10 noticias en noticias.json
✅ Completado.

